In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

%matplotlib inline

from pathlib import Path
import io
import urllib.parse as urlparse
import string
import random

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tqdm.auto import trange
from tqdm.auto import tqdm
import requests
from scrapy.http import TextResponse
from PIL import Image

In [2]:
# Load tokenizer and model

model_dir = Path("/home/hadim/.data/Neural_Network/captchanet/model/")
model_name = 'v6'
model_path = model_dir / model_name

with open(model_path / 'tokenizer.json') as f:
  #tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
  from keras_preprocessing import text
  tokenizer = text.tokenizer_from_json(f.read())
model = tf.keras.models.load_model(str(model_path / 'model'))

def captcha_solver(image):
    
  image = np.array(image)
  images = np.array([image])
  
  # The images need to be normalized t [0, 1] and resize to (224, 224).
  image_size = (224, 224)
  batch = tf.cast(images, 'float32')
  batch = tf.image.per_image_standardization(batch)
  batch = tf.image.resize(batch, image_size)

  # Run inference
  labels = model(batch)

  # Postprocess results (decode labels)
  labels = tf.argmax(labels, axis=2)
  words = tokenizer.sequences_to_texts(labels.numpy())
  words = [word.replace(' ', '') for word in words]
  
  return words[0]

In [3]:
root_url = "https://www.referendum.interieur.gouv.fr/consultation_publique/8/"

cookies_pot = [{'incap_ses_1226_2043128': 'Woh6D3J0pFOWdHEFuaADEewpKl0AAAAA+Kc92uBasp0Z3mffg9oR0g=='},
               {'incap_ses_1226_2043128': 'SIkeahIpgn7WmnEFuaADERAqKl0AAAAAn8UaMLoRiJBHGC+aDwscLA=='},
               {'incap_ses_1226_2043128': 'BjvoRw7WeABcvHEFuaADES8qKl0AAAAAD94H/YmkSl4aE+hfn0FAEQ=='},
               {'incap_ses_1226_2043128': 'uX9rcBMj0EJw7HEFuaADEV0qKl0AAAAAIdC8blGm87xsmBy9KDCk3g=='},
               {'incap_ses_1226_2043128': 'Ns3GUf+0hSXIFXIFuaADEYUqKl0AAAAAUNNWBbtHKaHGj1761Ffi7w=='}]

success = 0
rate = 0
word_length_max = 10
  
i = 0
pbar = tqdm()
while True:
  try:
  
    letter1 = random.choice(string.ascii_uppercase)
    letter2 = random.choice(string.ascii_uppercase)
    letter3 = random.choice(string.ascii_uppercase)
    url = urlparse.urljoin(root_url, f'{letter1}/{letter2}{letter3}')

    cookies = random.choice(cookies_pot)
    session = requests.Session()
    requests.utils.add_dict_to_cookiejar(session.cookies, cookies)

    req = session.get(url)
    response = TextResponse(str(req.url), body=req.text, encoding='utf-8')

    iframe_src = response.css('iframe').xpath('@src').get()
    if iframe_src and 'Incapsula' in iframe_src:
      raise Exception("Incapsula issue.")

    captcha = response.css('img#captcha').xpath('@src').get()

    if not captcha:
      raise Exception("Captcha not here.")

    # Get the token
    token = response.css('#form__token').xpath('@value').get()

    # Get the image captcha URL
    captcha_uri = response.css('img#captcha').xpath('@src').get()
    captcha_url = urlparse.urljoin(root_url, captcha_uri)

    # Download the image
    req = session.get(captcha_url)
    captcha_image = Image.open(io.BytesIO(req.content))

    # Solve the captcha
    captcha_solution = captcha_solver(captcha_image)

    # Send captcha solution.
    form_data = {}
    form_data['form[captcha]'] = captcha_solution
    form_data['form[_token]'] = token

    # Get the actual page.
    req = session.post(url, data=form_data)
    response = TextResponse(str(req.url), body=req.text, encoding='utf-8')

    iframe_src = response.css('iframe').xpath('@src').get()
    if iframe_src and 'Incapsula' in iframe_src:
      raise Exception("Incapsula issue.")

    captcha = response.css('img#captcha').xpath('@src').get()

    if not captcha:
      success += 1

    i += 1
    rate = (success / i) * 100
    text = f"{rate:.1f}% success rate ({success}/{i})"
    pbar.set_description(text)
    pbar.update()
    
  except ConnectionError:
    pass
  
pbar.close()

W0713 15:34:41.366332 139661934323520 deprecation.py:323] From /home/hadim/local/conda/envs/captchanet/lib/python3.7/site-packages/tensorflow/python/ops/image_ops_impl.py:1511: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


KeyboardInterrupt: 